In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.linear_model import Ridge

In [2]:
def trainModelTestTrainSplit(data, model):
    c = data.columns.difference(['Count'])
    X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(data[c], data['Count'], test_size=0.3)
    model.verbose=1
    model.fit(X_train.as_matrix(), y_train.as_matrix())
    pred = model.predict(X_test)
    print ("result ", mae(pred, y_test))
    return model

In [3]:
def trainModelKfold(data, model, countFold = 5):
    meanMae = 0
    
    c = data.columns.difference(['Count'])
    kf = KFold(len(data), n_folds=countFold, shuffle=True)
    
    for train_index, test_index in kf:
        X_train, X_test = data[c].iloc[train_index], data[c].iloc[test_index]
        y_train, y_test = data['Count'].iloc[train_index], data['Count'].iloc[test_index]
        model.fit(X_train.as_matrix(), y_train.as_matrix())
        pred = model.predict(X_test)
        print (mae(pred, y_test))
        meanMae += mae(pred, y_test)
    meanMae /= countFold
    print ("result ", meanMae)
    return model

In [4]:
features = pd.read_csv('data_transform/all.csv')
features.head(20)

,Count,IsTest,IsTrain,IsValidation,Prev1Day,Prev2Day,Prev3Day,Prev7Day,Next1Day,Date,...,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8.0,0,1,0,NaN,NaN,NaN,NaN,1.0,2011-10-06,...,85.75,96.584000,259.616162,226.261905,8.0,32.333333,32.333333,210.043716,85.75,219.600000
1,88.0,0,1,0,8.0,NaN,NaN,NaN,60.0,2011-10-07,...,85.75,88.366013,259.616162,250.045455,88.0,32.333333,32.333333,291.558011,85.75,157.333333
2,1.0,0,1,0,88.0,8.0,NaN,NaN,NaN,2011-10-09,...,85.75,144.766234,259.616162,247.447368,1.0,32.333333,32.333333,260.104972,85.75,222.000000
3,1.0,0,1,0,1.0,88.0,8.0,NaN,96.0,2011-11-15,...,85.75,126.602740,255.044944,232.555556,1.0,1.000000,1.000000,196.192513,85.75,93.000000
4,2.0,0,1,0,1.0,1.0,88.0,NaN,134.0,2011-12-08,...,85.75,70.965909,236.732673,253.739130,2.0,101.062500,18.333333,210.043716,85.75,141.000000
5,43.0,0,1,0,2.0,1.0,1.0,NaN,112.0,2011-12-09,...,85.75,77.677419,236.732673,247.447368,43.0,101.062500,18.333333,291.558011,85.75,183.800000
6,10.0,0,1,0,43.0,2.0,1.0,NaN,NaN,2011-12-10,...,85.75,87.256410,236.732673,222.318182,10.0,101.062500,18.333333,314.000000,85.75,180.000000
7,1.0,0,1,0,10.0,43.0,2.0,8.0,NaN,2011-12-11,...,85.75,110.950000,236.732673,241.594595,NaN,101.062500,18.333333,260.104972,85.75,244.000000
8,60.0,0,1,0,1.0,10.0,43.0,88.0,73.0,2011-12-13,...,85.75,130.255556,236.732673,224.358974,60.0,101.062500,92.000000,196.192513,85.75,214.000000
9,NaN,1,0,0,60.0,1.0,10.0,1.0,94.0,2011-12-14,...,85.75,127.515152,236.732673,231.727273,NaN,101.062500,92.000000,201.797688,85.75,302.500000


In [5]:
features.columns

Index([u'Count', u'IsTest', u'IsTrain', u'IsValidation', u'Prev1Day',
       u'Prev2Day', u'Prev3Day', u'Prev7Day', u'Next1Day', u'Date', u'Year',
       u'Month', u'MonthDay', u'WeekDay', u'Point', u'YearDay',
       u'ThroughMonth', u'ThroughWeek', u'ThroughDay', u'CityName', u'CityID',
       u'BranchNumber', u'MeanForMonth', u'MeanForMonthDay',
       u'MeanForThroughDay', u'MeanForThroughMonth', u'MeanForThroughWeek',
       u'MeanForWeekDay', u'MeanForYear', u'MeanForYearDay',
       u'MeanForMonthByPoints', u'MeanForMonthDayByPoints',
       u'MeanForThroughDayByPoints', u'MeanForThroughMonthByPoints',
       u'MeanForThroughWeekByPoints', u'MeanForWeekDayByPoints',
       u'MeanForYearByPoints', u'MeanForYearDayByPoints'],
      dtype='object')

In [6]:
features = pd.read_csv('data_transform/all.csv')
data = features.fillna(100000000)
#data = data[data.columns.difference(['Point', 'Date'])]

In [7]:
remove_column = ['IsTest', 'IsTrain', 'IsValidation', 'Date', 'Point', 'CityName', 'MeanForThroughDay', 'MeanForMonthDayByPoints' \
                 ,'MeanForThroughDayByPoints', 'MeanForThroughWeekByPoints', 'MeanForYearDayByPoints']

In [8]:
dataTrain = data[data.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

dataValidation =  data[data.IsValidation == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])]
y_test = dataValidation.Count

In [9]:
def trainModelKfoldOnValidation(data, model, countFold = 5):
    meanMae = 0
    
    c = data.columns.difference(['Count'])
    kf = KFold(len(data), n_folds=countFold, shuffle=True)
    
    for train_index, test_index in kf:
        X_train, X_test = data[c].iloc[train_index], data[c].iloc[test_index]
        y_train, y_test = data['Count'].iloc[train_index], data['Count'].iloc[test_index]
        model.fit(X_train.as_matrix(), y_train.as_matrix())
        pred = model.predict(X_test)
        print (mae(pred, y_test))
        meanMae += mae(pred, y_test)
    meanMae /= countFold
    print ("result ", meanMae)
    return model

In [38]:
model = trainModelTestTrainSplit(dataTrain, RandomForestRegressor())
pred = model.predict(X_test)
print (mae(pred, y_test))

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


('result ', 16.790921478453416)
17.5113636364


In [39]:
model = trainModelTestTrainSplit(dataTrain, RandomForestRegressor(100))
pred = model.predict(X_test)
print (mae(pred, y_test))

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   36.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


('result ', 16.241037301992847)
16.6029733642


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [15]:
model = trainModelTestTrainSplit(dataTrain, RandomForestRegressor(1000))
pred = model.predict(X_test)
print (mae(pred, y_test))

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  5.6min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    3.4s finished


('result ', 16.1461769715551)
16.4290610886


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.2s finished


In [7]:
remove_column = ['IsTest', 'IsTrain', 'IsValidation', 'Date', 'CityName', 'MeanForThroughDay', 'MeanForMonthDayByPoints' \
                 ,'MeanForThroughDayByPoints', 'MeanForThroughWeekByPoints', 'MeanForYearDayByPoints']

In [8]:
data = features.fillna(100000000)

In [9]:
data = features.fillna(100000000)

dataTrain = data[data.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

X_train = dataTrain[dataTrain.columns.difference(['Count'])]
y_train = dataTrain[['Count', 'Point']]

dataValidation =  data[data.IsValidation == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])]
y_test = dataValidation[['Count', 'Point']]

In [259]:
def FitOnCitySeparate(model):
    city = features.Point.unique()
    pred = dataValidation.Count


    for i in city:
        X = X_train[X_train.Point == i]
        X = X[X.columns.difference(['Point'])]

        y = y_train[y_train.Point == i].Count

        X_val = X_test[X_test.Point == i]
        X_val = X_val[X_val.columns.difference(['Point'])]

        y_val = y_test[y_test.Point == i].Count

        model.fit(X, y)

        pred[y_val.index] = model.predict(X_val)
        print i,
    
    pred = pred.apply(lambda x:  1 if x < 1 else x)
    #pred = pred.apply(lambda x:  round(x) if abs(x - round(x)) < 0.4 else x)
    print ""
    print mae(y_test.Count, pred)
    return pred

In [261]:
y_test["predict"] = pred

In [222]:
FitOnCitySeparate(GradientBoostingRegressor(learning_rate=0.1, max_depth=7, n_estimators=300, loss ='lad'))

Сыктывкар-1 Сыктывкар-2 Ухта-1 Смоленск-1 Сарапул-1 Самара-1 Челябинск-1 Белгород-1 Смоленск-2 Самара-2 Ульяновск-1 Сыктывкар-3 Октябрьский-1 Сыктывкар-4 Долгопрудный-1 Таганрог-1 Энгельс-1 Красноярск-1 Абакан-1 Ульяновск-2 Майкоп-1 Самара-3 Нижнекамск-1 Королев-1 Ростов-на-Дону-1 Альметьевск-1 Гатчина-1 Смоленск-3 Химки-1 Махачкала-1 Тюмень-1 Бугульма-1 Уссурийск-1 Сыктывкар-5 Самара-4 Колпино-1 Астрахань-1 Новороссийск-1 Мытищи-1 Ачинск-1 Волжский-1 Муром-1 Калининград-1 Березники-1 Ростов-на-Дону-2 Чайковский-1 Нижневартовск-1 Саратов-1 Реутов-1 Одинцово-1 Комсомольск-на-Амуре-1 Домодедово-1 Вологда-1 Балашиха-1 Ангарск-1 Сыктывкар-6 Сургут-1 Нахабино-1 Калуга-1 Щелково-1 Гатчина-2 Зеленоград-1 Всеволожск-1 Новочебоксарск-1 Железнодорожный-1 Москва 1-1 Шахты-1 Салават-1 Челябинск-2 Ленинск-Кузнецкий-1 Лобня-1 Ханты-Мансийск-1 Балаково-1 Тверь-1 Тольятти-1 Пушкино-1 Калининград-2 Нефтекамск-1 Петрозаводск-1 Сочи-1 Казань-1 Вельск-1 Выборг-1 Тобольск-1 Ухта-2 Абинск-1 Нижний Новгород-

In [275]:
pred = FitOnCitySeparate(GradientBoostingRegressor(learning_rate=0.1, max_depth=2, n_estimators=100, loss ='lad', criterion = 'mae'))

Сыктывкар-1 Сыктывкар-2 Ухта-1 Смоленск-1 Сарапул-1 Самара-1 Челябинск-1 Белгород-1 Смоленск-2 Самара-2 Ульяновск-1 Сыктывкар-3 Октябрьский-1 Сыктывкар-4 Долгопрудный-1 Таганрог-1 Энгельс-1 Красноярск-1 Абакан-1 Ульяновск-2 Майкоп-1 Самара-3 Нижнекамск-1 Королев-1 Ростов-на-Дону-1 Альметьевск-1 Гатчина-1 Смоленск-3 Химки-1 Махачкала-1 Тюмень-1 Бугульма-1 Уссурийск-1 Сыктывкар-5 Самара-4 Колпино-1 Астрахань-1 Новороссийск-1 Мытищи-1 Ачинск-1 Волжский-1 Муром-1 Калининград-1 Березники-1 Ростов-на-Дону-2 Чайковский-1 Нижневартовск-1 Саратов-1 Реутов-1 Одинцово-1 Комсомольск-на-Амуре-1 Домодедово-1 Вологда-1 Балашиха-1 Ангарск-1 Сыктывкар-6 Сургут-1 Нахабино-1 Калуга-1 Щелково-1 Гатчина-2 Зеленоград-1 Всеволожск-1 Новочебоксарск-1 Железнодорожный-1 Москва 1-1 Шахты-1 Салават-1 Челябинск-2 Ленинск-Кузнецкий-1 Лобня-1 Ханты-Мансийск-1 Балаково-1 Тверь-1 Тольятти-1 Пушкино-1 Калининград-2 Нефтекамск-1 Петрозаводск-1 Сочи-1 Казань-1 Вельск-1 Выборг-1 Тобольск-1 Ухта-2 Абинск-1 Нижний Новгород-

In [11]:
pred = FitOnCitySeparate(GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=100, loss ='lad', criterion = 'mae'))

NameError: name 'FitOnCitySeparate' is not defined

In [175]:
for i in city:
    X = X_train[X_train.Point == i]
    X = X[X.columns.difference(['Point'])]
    
    y = y_train[y_train.Point == i].Count
    
    X_val = X_train[X_train.Point == i]
    X_val = X_val[X_val.columns.difference(['Point'])]
    
    y_val = y_train[y_train.Point == i].Count
    
    model.fit(X, y)
    predict = model.predict(X_val)
    k = 0
    print y_val.index.shape
    print y_val.index
    for i in y_val.index:
        pass
        #print i
        #pred.Count[i] = predict[k]
        #k += 1
    break

(1265,)
Int64Index([    0,     1,     2,     3,     5,     6,     8,    10,    11,
               12,
            ...
            52185, 52275, 52462, 52570, 52696, 52823, 52827, 53079, 53194,
            53377],
           dtype='int64', length=1265)


In [17]:
data.columns

Index([u'Count', u'IsTest', u'IsTrain', u'IsValidation', u'Prev1Day',
       u'Prev2Day', u'Prev3Day', u'Prev7Day', u'Next1Day', u'Next2Day',
       u'Next3Day', u'Next7Day', u'Date', u'Year', u'Month', u'MonthDay',
       u'WeekDay', u'Point', u'YearDay', u'ThroughMonth', u'ThroughWeek',
       u'ThroughDay', u'CityName', u'CityID', u'BranchNumber', u'MeanForMonth',
       u'MeanForMonthDay', u'MeanForThroughDay', u'MeanForThroughMonth',
       u'MeanForThroughWeek', u'MeanForWeekDay', u'MeanForYear',
       u'MeanForYearDay', u'MeanForMonthByPoints', u'MeanForMonthDayByPoints',
       u'MeanForThroughDayByPoints', u'MeanForThroughMonthByPoints',
       u'MeanForThroughWeekByPoints', u'MeanForWeekDayByPoints',
       u'MeanForYearByPoints', u'MeanForYearDayByPoints'],
      dtype='object')

In [10]:
features = pd.read_csv('data_transform/all.csv')

data = features.fillna(100000000)
#data = features
remove_column = ['IsTest', 'IsTrain', 'IsValidation', 'Date', 'CityName', 'MeanForThroughDay', 'MeanForMonthDayByPoints' \
                 ,'MeanForThroughDayByPoints', 'MeanForThroughWeekByPoints', 'MeanForYearDayByPoints', 'Point']
       #, u'Prev1Day',\
       #u'Prev2Day', u'Prev3Day', u'Prev7Day', u'Next1Day', u'Next2Day',
       #u'Next3Day', u'Next7Day']

dataTrain = data[data.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

X_train = dataTrain[dataTrain.columns.difference(['Count'])]
y_train = dataTrain[['Count']]

dataValidation =  data[data.IsValidation == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])]
y_test = dataValidation[['Count']]

In [27]:
gbr = GradientBoostingRegressor(learning_rate=0.01, max_depth=7, verbose=1, n_estimators=5000, loss ='lad')

In [28]:
gbr.fit(X_train, y_train)

/home/laol/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1          52.2156           19.84m
         2          51.7924           19.67m
         3          51.3738           19.58m
         4          50.9568           19.70m
         5          50.5456           19.62m
         6          50.1284           19.76m
         7          49.7193           19.80m
         8          49.3230           19.86m
         9          48.9227           19.86m
        10          48.5418           19.87m
        20          44.8297           19.47m
        30          41.5776           19.53m
        40          38.6103           19.51m
        50          35.9221           19.38m
        60          33.5036           19.25m
        70          31.3448           19.18m
        80          29.4390           19.26m
        90          27.6988           19.28m
       100          26.1602           19.28m
       200          17.9053           19.53m
       300          15.4996           18.51m
       40

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.01,
             loss='lad', max_depth=7, max_features=None,
             max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=5000,
             presort='auto', random_state=None, subsample=1.0, verbose=1,
             warm_start=False)

In [29]:
pred = gbr.predict(X_test)
print mae(y_test, pred)

14.888028891


In [49]:
gbr = RandomForestRegressor(100)
gbr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [50]:
pred = gbr.predict(X_test)
print mae(y_test, pred)

15.9456181239


In [12]:
gbr = GradientBoostingRegressor(learning_rate=0.01, max_depth=7, verbose=1, n_estimators=1000, loss ='lad')

In [13]:
gbr.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1          52.1785            5.01m
         2          51.7565            5.03m
         3          51.3385            5.14m
         4          50.9242            5.23m
         5          50.5166            5.21m
         6          50.1100            5.20m
         7          49.7070            5.27m
         8          49.3077            5.36m
         9          48.9126            5.37m
        10          48.5183            5.33m
        20          44.8075            5.11m
        30          41.5091            4.98m
        40          38.5191            4.89m
        50          35.8354            4.85m
        60          33.4426            4.80m
        70          31.2776            4.75m
        80          29.3617            4.72m
        90          27.6607            4.66m
       100          26.1375            4.61m
       200          17.8064            4.01m
       300          15.3503            3.42m
       40

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='lad', max_depth=7,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=1, warm_start=False)

In [14]:
pred = gbr.predict(X_test)
print mae(y_test, pred)

99999896.6248


In [15]:
gbr.feature_importances_

array([ 0.00834371,  0.0257306 ,  0.01317664,  0.03274509,  0.02681527,
        0.01518023,  0.25115608,  0.03398799,  0.02719227,  0.06621925,
        0.00186129,  0.02466117,  0.06764238,  0.00208313,  0.02951469,
        0.04409268,  0.03073785,  0.02924413,  0.03198458,  0.0413409 ,
        0.02730942,  0.02859148,  0.03532443,  0.02246023,  0.0024261 ,
        0.00689398,  0.02553902,  0.00046523,  0.04728016])

In [81]:
X_test.columns

Index([u'BranchNumber', u'CityID', u'MeanForMonth', u'MeanForMonthByPoints',
       u'MeanForMonthDay', u'MeanForThroughMonth',
       u'MeanForThroughMonthByPoints', u'MeanForThroughWeek',
       u'MeanForWeekDay', u'MeanForWeekDayByPoints', u'MeanForYear',
       u'MeanForYearByPoints', u'MeanForYearDay', u'Month', u'MonthDay',
       u'Next1Day', u'Next2Day', u'Next3Day', u'Prev1Day', u'Prev2Day',
       u'Prev3Day', u'ThroughDay', u'ThroughMonth', u'ThroughWeek', u'WeekDay',
       u'Year', u'YearDay'],
      dtype='object')

In [57]:
gbr = GradientBoostingRegressor(learning_rate=0.01, max_depth=7, verbose=1, n_estimators=1000, loss ='lad')

In [58]:
gbr.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1          51.9711            4.16m
         2          51.5338            3.99m
         3          51.1044            3.93m
         4          50.6769            3.88m
         5          50.2535            3.87m
         6          49.8344            3.86m
         7          49.4193            3.86m
         8          49.0089            3.86m
         9          48.6062            3.88m
        10          48.2046            3.91m
        20          44.4241            3.86m
        30          41.0447            3.84m
        40          38.0405            3.77m
        50          35.3418            3.74m
        60          32.9457            3.70m
        70          30.8143            3.65m
        80          28.9158            3.61m
        90          27.2273            3.56m
       100          25.7372            3.56m
       200          17.8235            3.12m
       300          15.5688            2.63m
       40

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='lad', max_depth=7,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=1, warm_start=False)

In [59]:
pred = gbr.predict(X_test)
print mae(y_test, pred)

15.2123040754


In [28]:
gbr = GradientBoostingRegressor(learning_rate=0.01, max_depth=7, verbose=1, n_estimators=4000, loss ='lad')

In [29]:
gbr.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1          52.0946           14.51m
         2          51.6626           15.34m
         3          51.2344           15.78m
         4          50.8138           15.91m
         5          50.3961           16.01m
         6          49.9769           15.94m
         7          49.5602           15.86m
         8          49.1497           16.21m
         9          48.7457           16.01m
        10          48.3411           15.88m
        20          44.5455           14.98m
        30          41.1503           14.61m
        40          38.1413           14.39m
        50          35.4368           14.27m
        60          33.0354           14.19m
        70          30.8824           14.14m
        80          28.9630           14.08m
        90          27.2523           14.06m
       100          25.7310           14.08m
       200          17.7504           13.48m
       300          15.5231           12.72m
       40

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='lad', max_depth=7,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=4000, presort='auto', random_state=None,
             subsample=1.0, verbose=1, warm_start=False)

In [30]:
pred = gbr.predict(X_test)
print mae(y_test, pred)

15.42651794


In [19]:
model = trainModelKfoldOnValidation(dataTrain, RandomForestRegressor(), countFold = 25)
pred = model.predict(X_test)
print (mae(pred, y_test))

16.632610083
18.7105296745
18.443905552
16.315252074
16.8954690491
17.0089342693
16.1523292916
17.0751754946
17.5008296107
16.970134014
17.7065092534
16.8759412891
15.4571155073
16.1772814295
16.3549457562
16.8608806637
16.846330568
17.8841097639
16.939948947
17.321952776
16.6019783025
16.478366305
16.4514358647
16.62744097
16.6901723038
result  16.9191831525
16.875267573


In [ ]:
m = trainModelKfold(data, LinearRegression(), countFold = 25)
print(data.columns.difference(['Count']))
m.coef_

# Weather (be or not to be)

In [100]:
features = pd.read_csv('data_transform/all.csv')
remove_column = ['IsTest', 'IsTrain', 'IsValidation', 'lat', 'lng', 'FirstOrder', 'BranchNumber', 'CityID', 'Date', 'Point', 'CityName', 'MeanForThroughDay', 'MeanForMonthDayByPoints' \
                 ,'MeanForThroughDayByPoints', 'MeanForThroughWeekByPoints', 'MeanForYearDayByPoints']

/opt/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## without weather

In [96]:
weather = pd.read_csv('data_transform/tmp_weather.csv')
weather = weather[pd.notnull(weather.summary)]
weather.head()

/opt/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,CityName,lat,lng,summary,icon,precipType,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windBearing,visibility,cloudCover,pressure
0,2011-10-06,Сыктывкар,61.647851,50.833903,Overcast,cloudy,rain,4.95,1.77,4.13,0.94,4.05,195.0,8.32,1.0,990.66
1,2011-10-07,Сыктывкар,61.647851,50.833903,Overcast,cloudy,rain,7.22,5.09,5.04,0.86,3.13,240.0,11.27,1.0,NaN
2,2011-10-09,Сыктывкар,61.647851,50.833903,Partly Cloudy,partly-cloudy-night,rain,9.71,9.00,8.08,0.90,1.82,229.0,8.48,0.5,1007.93
3,2011-11-15,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-1.04,-5.66,-1.32,0.98,4.14,205.0,5.63,1.0,988.27
4,2011-12-08,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-1.78,-3.57,-2.19,0.97,1.38,171.0,10.54,1.0,1013.80


In [97]:
#column with text to number
uniq = weather.summary.unique()
weather.summary = weather.summary.apply(lambda x: np.where(uniq == x)[0][0]) 

uniq = weather.icon.unique()
weather.icon = weather.icon.apply(lambda x: np.where(uniq == x)[0][0]) 

uniq = weather.precipType.unique()
weather.precipType = weather.precipType.apply(lambda x: np.where(uniq == x)[0][0]) 

weather.head()

,Date,CityName,lat,lng,summary,icon,precipType,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windBearing,visibility,cloudCover,pressure
0,2011-10-06,Сыктывкар,61.647851,50.833903,0,0,0,4.95,1.77,4.13,0.94,4.05,195.0,8.32,1.0,990.66
1,2011-10-07,Сыктывкар,61.647851,50.833903,0,0,0,7.22,5.09,5.04,0.86,3.13,240.0,11.27,1.0,NaN
2,2011-10-09,Сыктывкар,61.647851,50.833903,1,1,0,9.71,9.00,8.08,0.90,1.82,229.0,8.48,0.5,1007.93
3,2011-11-15,Сыктывкар,61.647851,50.833903,0,0,1,-1.04,-5.66,-1.32,0.98,4.14,205.0,5.63,1.0,988.27
4,2011-12-08,Сыктывкар,61.647851,50.833903,0,0,1,-1.78,-3.57,-2.19,0.97,1.38,171.0,10.54,1.0,1013.80


In [101]:
indexes = weather.index
with_weather = features.iloc[indexes]

dataTrain = with_weather[with_weather.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

X_train = dataTrain[dataTrain.columns.difference(['Count'])].as_matrix()
y_train = dataTrain.Count.as_matrix()

dataValidation =  with_weather[with_weather.IsValidation == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])].as_matrix()
y_test = dataValidation.Count.as_matrix()
dataTrain

,Count,MeanForMonth,MeanForMonthByPoints,MeanForMonthDay,MeanForThroughMonth,MeanForThroughMonthByPoints,MeanForThroughWeek,MeanForWeekDay,MeanForWeekDayByPoints,MeanForYear,MeanForYearByPoints,MeanForYearDay,Month,MonthDay,ThroughDay,ThroughMonth,ThroughWeek,WeekDay,Year,YearDay
0,8.0,112.145788,259.317757,104.137491,32.333333,32.333333,32.333333,90.875335,214.426230,87.318182,87.318182,94.327731,10,6,0,0,0,3,2011,278
1,88.0,112.145788,259.317757,107.172263,32.333333,32.333333,32.333333,123.721962,291.581522,87.318182,87.318182,91.530201,10,7,1,0,0,4,2011,279
2,1.0,112.145788,259.317757,107.880699,32.333333,32.333333,32.333333,123.640722,262.022099,87.318182,87.318182,143.346821,10,9,3,0,0,6,2011,281
3,1.0,104.213605,257.108696,108.351462,1.000000,1.000000,1.000000,87.286115,198.868852,87.318182,87.318182,124.594937,11,15,40,1,6,1,2011,318
5,43.0,105.650704,230.669643,107.880699,101.277778,101.277778,18.000000,123.721962,291.581522,87.318182,87.318182,72.645161,12,9,64,2,9,4,2011,342
6,10.0,105.650704,230.669643,102.119765,101.277778,101.277778,18.000000,142.693482,312.527174,87.318182,87.318182,81.650000,12,10,65,2,9,5,2011,343
7,1.0,105.650704,230.669643,102.062201,101.277778,101.277778,18.000000,123.640722,262.022099,87.318182,87.318182,103.202532,12,11,66,2,9,6,2011,344
8,60.0,105.650704,230.669643,100.133333,101.277778,101.277778,100.400000,87.286115,198.868852,87.318182,87.318182,122.585106,12,13,68,2,10,1,2011,346
10,96.0,105.650704,230.669643,108.351462,101.277778,101.277778,100.400000,90.875335,214.426230,87.318182,87.318182,81.652632,12,15,70,2,10,3,2011,348
11,134.0,105.650704,230.669643,103.290323,101.277778,101.277778,100.400000,123.721962,291.581522,87.318182,87.318182,77.175824,12,16,71,2,10,4,2011,349


In [102]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print (mae(pred, y_test))

17.9631818182


In [110]:
weather_columns = weather.columns.difference(['Date', 'CityName', 'lat', 'lng'])
with_weather[weather_columns] = weather[weather_columns]
with_weather
with_weather.fillna(method='bfill', inplace=True)
with_weather.fillna(method='ffill', inplace=True)
dataTrain = with_weather[with_weather.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

X_train = dataTrain[dataTrain.columns.difference(['Count'])].as_matrix()
y_train = dataTrain.Count.as_matrix()

dataValidation =  with_weather[with_weather.IsValidation == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])].as_matrix()
y_test = dataValidation.Count.as_matrix()
dataTrain

/opt/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/opt/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Count,MeanForMonth,MeanForMonthByPoints,MeanForMonthDay,MeanForThroughMonth,MeanForThroughMonthByPoints,MeanForThroughWeek,MeanForWeekDay,MeanForWeekDayByPoints,MeanForYear,...,dewPoint,humidity,icon,precipType,pressure,summary,temperature,visibility,windBearing,windSpeed
0,8.0,112.145788,259.317757,104.137491,32.333333,32.333333,32.333333,90.875335,214.426230,87.318182,...,4.13,0.94,0,0,990.66,0,4.95,8.32,195.0,4.05
1,88.0,112.145788,259.317757,107.172263,32.333333,32.333333,32.333333,123.721962,291.581522,87.318182,...,5.04,0.86,0,0,1007.93,0,7.22,11.27,240.0,3.13
2,1.0,112.145788,259.317757,107.880699,32.333333,32.333333,32.333333,123.640722,262.022099,87.318182,...,8.08,0.90,1,0,1007.93,1,9.71,8.48,229.0,1.82
3,1.0,104.213605,257.108696,108.351462,1.000000,1.000000,1.000000,87.286115,198.868852,87.318182,...,-1.32,0.98,0,1,988.27,0,-1.04,5.63,205.0,4.14
5,43.0,105.650704,230.669643,107.880699,101.277778,101.277778,18.000000,123.721962,291.581522,87.318182,...,-4.04,0.89,0,1,1026.65,0,-2.44,9.06,188.0,0.89
6,10.0,105.650704,230.669643,102.119765,101.277778,101.277778,18.000000,142.693482,312.527174,87.318182,...,-6.47,0.86,0,1,1026.66,0,-4.43,10.61,163.0,2.45
7,1.0,105.650704,230.669643,102.062201,101.277778,101.277778,18.000000,123.640722,262.022099,87.318182,...,-8.93,0.88,0,1,1018.29,0,-7.28,8.42,173.0,4.10
8,60.0,105.650704,230.669643,100.133333,101.277778,101.277778,100.400000,87.286115,198.868852,87.318182,...,-11.34,0.88,0,1,1017.16,0,-9.77,8.40,192.0,3.49
10,96.0,105.650704,230.669643,108.351462,101.277778,101.277778,100.400000,90.875335,214.426230,87.318182,...,-11.45,0.84,0,1,1024.86,0,-9.23,10.80,170.0,3.61
11,134.0,105.650704,230.669643,103.290323,101.277778,101.277778,100.400000,123.721962,291.581522,87.318182,...,-6.72,0.94,0,1,1022.50,0,-5.90,7.19,162.0,2.77


In [111]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print (mae(pred, y_test))


17.9131818182


In [115]:
f = pd.DataFrame()
f['val'] = model.feature_importances_
f['Name'] = dataTrain.columns.difference(['Count'])
f.sort_values(by='val')

,val,Name
24,0.000083,precipType
17,0.000110,Year
23,0.000277,icon
8,0.000320,MeanForYear
3,0.000366,MeanForThroughMonth
26,0.000548,summary
11,0.000636,Month
0,0.000872,MeanForMonth
1,0.001005,MeanForMonthByPoints
21,0.001240,dewPoint
